Heap

How to heapify without using build-in heapq

In [ ]:
'''
[5,2,3,1] -> [1,2,3,5]

1) Heapify: bottom - down + sift down (parent and children)
2) Sort: swap larger in index 0 with last element
'''
from typing import List
class Solution:
    def sortArray(self, nums: List[int]) -> List[int]:
        n = len(nums)

        # heapify
        self.heapify(nums)

        # swap larger with last element
        for i in range(n-1, 0, -1):
            # swap largest heap in root with last element of heap
            nums[0], nums[i] = nums[i], nums[0]

            self.sift_down(nums, 0, i)

        return nums

    def heapify(self, heap):
        n = len(heap)
        # non leaf
        non_leaf = n // 2 - 1

        # scan bottom up
        for i in range(non_leaf, -1, -1):
            self.sift_down(heap, i, n)

    def sift_down(self, heap, parent, size_heap):
        # index updates
        largest = parent
        left = parent * 2 + 1
        right = parent * 2 + 2

        # check larger between children and parents
        if left < size_heap and heap[largest] < heap[left]:
            largest = left
        if right < size_heap and heap[largest] < heap[right]:
            largest = right

        # swap if larger is children 
        if largest != parent:
            heap[largest], heap[parent] = heap[parent], heap[largest]
            # sift down for new largset
            self.sift_down(heap, largest, size_heap)

Heap - K top element

In [ ]:
from typing import List 
from heapq import heappush, heappop
from math import sqrt

class Solution:
    '''
    Here N refers to the length of the given array points.

    Time complexity: O(N⋅logk)

    Adding to/removing from the heap (or priority queue) only takes O(logk) time when the size of the heap is capped at k elements.

    Space complexity: O(k)

    The heap (or priority queue) will contain at most k elements.
    '''
    def kClosest(self, points: List[List[int]], k: int) -> List[List[int]]:
        # distance sqrt(x**2 + y**2) -> smallest
        def calculateDistance(x,y):
            return sqrt(x**2+y**2)
        
        max_heap = []
        for x, y in points:
            # distance
            distance = calculateDistance(x,y)
            
            heappush(max_heap, (-distance, [x, y]))

            # remove until k 
            if len(max_heap) > k:
                heappop(max_heap)

        return [coordinate for _, coordinate in max_heap]

Priority q - frequency - k internal between elements

Task scheduler - CPU tasks

In [ ]:
from heapq import heappush, heappop
from collections import Counter
from typing import List

class Solution:
    '''
    Tasks: ['A', 'A', 'A', 'B', 'B', 'B'], n = 2
    Max Heap: [-3, -3] (negated values for max heap behavior).
    Push back tasks to heap: [-2, -2].
    Push back tasks to heap: [-1, -1].

    Time: 1 → Execute A, decrement to -2.

    Time: 2 → Execute B, decrement to -2.

    Time: 3 → Idle (no tasks available for cooldown).

    Push back tasks to heap: [-2, -2].

    Time: 4 → Execute A, decrement to -1.

    Time: 5 → Execute B, decrement to -1.

    Time: 6 → Idle.

    Push back tasks to heap: [-1, -1].

    Time: 7 → Execute A, decrement to 0.

    Time: 8 → Execute B, decrement to 0.

    Time Complexity: O(m)+O(klogk)+O(n⋅logk) -> maximum 26 unique character (constant time) -> O(m) + O(1)
        - m: Total number of tasks.
        - k: Number of unique tasks.
        - n: Cooldown interval.

        1. count frequency dictionary: O(m)
        2. build heap: O(klogk)
        3. Inner loops:
            - heap operation: O((n+1).logk = nlogk
            - reinsert remaining tasks: O(klogk)
        Total: O(m)+O(klogk)+O(n⋅logk)

    Space Complexity: O(k+n) 
        - O(k) for the heap (constant 26) 
        - additional storage O(n). 
    '''
    def leastInterval(self, tasks: List[str], n: int) -> int:
        # priotity Queue
        # Output: time
        # cooldown window: n
        time = 0

        # dictionary counter
        task_counter = Counter(tasks)

        # create a max heap for counters
        max_heap = []
        for counter in task_counter.values():
            heappush(max_heap, -counter)
        
        # Count timer in cooldown window 
        while max_heap:
            # record counters
            record_remaining_counter = []
            # add timer in cooldown window - include idle by adding to time for 0-n (include n)
            for _ in range(n+1):
                # there is tasks of character in max heap
                # max heap
                if max_heap:
                    count = heappop(max_heap)
                    # if there is remaining tasks
                    if count < -1:
                        # decrement counter
                        record_remaining_counter.append(count + 1)
                # there is idle and nothing in max heap
                time += 1

                # no max heap
                if not max_heap and not record_remaining_counter:
                    break

            # add remaining to max heap
            for count in record_remaining_counter:
                heappush(max_heap, count)

        return time

Reorgonize String - Priority Q

In [ ]:
from collections import Counter
from heapq import heappush, heappop
class Solution:
    '''
    count frequency + heap (priotity queue)

    Time: nlogk = n
    - n: s length
    - k: unique character
    - logk: 26 character

    Space: n + k
    '''
    def reorganizeString(self, s: str) -> str:
        # 4a 3b -> 
        # i != i+1
        # counter -1
        # distance k - skip only 1 time
                    # in next loop, push it 
        
        # count frequency
        count_frequency = Counter(s)

        # construct a max heap
        max_heap = []
        for char, count in count_frequency.items(): 
            heappush(max_heap, (-count, char))

        # reorgonize based on priority and skip 1 loop for same character by removing and adding in next loop
        result = []
        pre_freq, pre_char = 0,""
        while max_heap:
            # give me the most frequent char
            freq, char = heappop(max_heap)
            result.append(char)

            # add previous char with freq < 0 - this line is one loop behind -> never add same character to result
            if pre_freq < 0:
                heappush(max_heap, (pre_freq, pre_char))

            pre_freq, pre_char = freq + 1, char

        return "".join(result) if len(s) == len(result) else ""